In [1]:
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import trange
from torch.autograd import Variable

In [2]:
import gensim.downloader
word2vec_goog1e_news: gensim.models.keyedvectors.KeyedVectors = gensim.downloader.load('word2vec-google-news-300')
word2vec_goog1e_news.add_vector("<pad>", np.zeros(300))
pad_index = word2vec_goog1e_news.key_to_index["<pad>"]
embedding_weights = torch.FloatTensor(word2vec_goog1e_news.vectors)
vocab = word2vec_goog1e_news.key_to_index
# UNK is already inside word2vec_google_news
# weights = torch.FloatTensor(word2vec_goog1e_news.vectors)
# embedding = nn.Embedding.from_pretrained(weights)
# WORD = 'UNK'
# WORDID = word2vec_goog1e_news.key_to_index[WORD]
# print(f"WORDID: {WORDID}")

c:\Users\jerem\OneDrive\Documents\NLP_Assignment\SC4002_NLP\venv\lib\site-packages\gensim\models\keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


In [3]:
def read_conll_file(file_path):
    sentences = []
    sentence = []
    tags = []
    tag = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if len(line) == 0:  # Empty line indicates the end of a sentence
                if sentence:
                    sentences.append(sentence)
                    tags.append(tag)
                sentence = []
                tag = []
            else:
                parts = line.split()
                word = parts[0]
                if word == '-DOCSTART-':
                    continue
                ner_tag = parts[-1]
                sentence.append(word)
                tag.append(ner_tag)
        if sentence:
            sentences.append(sentence)
            tags.append(tag)
    return sentences, tags

In [4]:
train_data, train_tags = read_conll_file('CoNLL2003_dataset/eng.train')
val_data, val_tags = read_conll_file('CoNLL2003_dataset/eng.testa')
test_data, test_tags = read_conll_file('CoNLL2003_dataset/eng.testb')

In [5]:
from itertools import chain

def get_unique_words(allSentences):
    uniqueWords = [] 
    flatten_sentences = list(chain(*allSentences))
    for i in flatten_sentences:
        if not i in uniqueWords:
            uniqueWords.append(i);
    uniqueWords.append('<pad>')
    uniqueWords.append('UNK')
    return uniqueWords

def get_unique_tags(allTags):
    uniqueWords = []
    flatten_sentences = list(chain(*allTags))
    for i in flatten_sentences:
        if not i in uniqueWords:
            uniqueWords.append(i);
    return uniqueWords

In [6]:
unique_words = get_unique_words(train_data+val_data) # test_data not included
unique_tags = get_unique_tags(train_tags+val_tags+test_tags)

In [7]:
tag_map = {}
for index, tag in enumerate(unique_tags):
    tag_map[tag] = index
tag_map

{'I-ORG': 0,
 'O': 1,
 'I-MISC': 2,
 'I-PER': 3,
 'I-LOC': 4,
 'B-LOC': 5,
 'B-MISC': 6,
 'B-ORG': 7}

In [8]:
def indexify(data, tag):
    setences = []
    labels = []
    for sentence in data:
        s = [vocab[token] if token in vocab
            else vocab['UNK']
            for token in sentence]
        setences.append(s)

    for sentence in tag:
        l = [tag_map[label] for label in sentence]
        labels.append(l)

    return setences, labels

train_sentences = []
train_labels = []

val_sentences = []
val_labels = []

test_sentences = []
test_labels = []

train_sentences, train_labels = indexify(train_data, train_tags)
val_sentences, val_labels = indexify(val_data, val_tags)
test_sentences, test_labels = indexify(test_data, test_tags)

In [9]:
# print(f"train_sentences: \n {train_sentences}")
# print(f"train_labels: \n {train_labels}")

In [10]:
def data_iterator(sentences, labels, total_size: int, batch_size: int, shuffle: bool=False):
    # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
    order = list(range(total_size))
    if shuffle:
        random.seed(230)
        random.shuffle(order)

    # one pass over data
    for i in range((total_size+1)//batch_size):
        # fetch sentences and tags
        batch_sentences = [sentences[idx] for idx in order[i*batch_size:(i+1)*batch_size]]
        batch_tags = [labels[idx] for idx in order[i*batch_size:(i+1)*batch_size]]

        # compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in batch_sentences])

        # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
        # initialising labels to -1 differentiates tokens with tags from PADding tokens
        batch_data = vocab['<pad>']*np.ones((len(batch_sentences), batch_max_len))
        batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))

        # copy the data to the numpy array
        for j in range(len(batch_sentences)):
            cur_len = len(batch_sentences[j])
            batch_data[j][:cur_len] = batch_sentences[j]
            batch_labels[j][:cur_len] = batch_tags[j]

        # since all data are indices, we convert them to torch LongTensors
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)

        # shift tensors to GPU if available
        # if params.cuda:
        #     batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

        # convert them to Variables to record operations in the computational graph
        batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)

        yield batch_data, batch_labels, batch_sentences

In [11]:
class Net(nn.Module):
    """
    This is the standard way to define your own network in PyTorch. You typically choose the components
    (e.g. LSTMs, linear layers etc.) of your network in the __init__ function. You then apply these layers
    on the input step-by-step in the forward function. You can use torch.nn.functional to apply functions
    such as F.relu, F.sigmoid, F.softmax. Be careful to ensure your dimensions are correct after each step.

    You are encouraged to have a look at the network in pytorch/vision/model/net.py to get a better sense of how
    you can go about defining your own network.

    The documentation for all the various components available to you is here: http://pytorch.org/docs/master/nn.html
    """

    def __init__(self, embedding_weights, embedding_dim, lstm_hidden_dim, number_of_tags):
        """
        We define an recurrent network that predicts the NER tags for each token in the sentence. The components
        required are:

        - an embedding layer: this layer maps each index in range(params.vocab_size) to a params.embedding_dim vector
        - lstm: applying the LSTM on the sequential input returns an output for each token in the sentence
        - fc: a fully connected layer that converts the LSTM output for each token to a distribution over NER tags

        Args:
            params: (Params) contains vocab_size, embedding_dim, lstm_hidden_dim
        """
        super(Net, self).__init__()

        # the embedding takes as input the vocab_size and the embedding_dim
        # self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, padding_idx=pad_index)

        # the LSTM takes as input the size of its input (embedding_dim), its hidden size
        # for more details on how to use it, check out the documentation
        self.lstm = nn.LSTM(embedding_dim,
                            lstm_hidden_dim, batch_first=True)

        # the fully connected layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, number_of_tags)

    def forward(self, s):
        """
        This function defines how we use the components of our network to operate on an input batch.

        Args:
            s: (Variable) contains a batch of sentences, of dimension batch_size x seq_len, where seq_len is
               the length of the longest sentence in the batch. For sentences shorter than seq_len, the remaining
               tokens are PADding tokens. Each row is a sentence with each element corresponding to the index of
               the token in the vocab.

        Returns:
            out: (Variable) dimension batch_size*seq_len x num_tags with the log probabilities of tokens for each token
                 of each sentence.

        Note: the dimensions after each step are provided
        """
        #                                -> batch_size x seq_len
        # apply the embedding layer that maps each token to its embedding
        # dim: batch_size x seq_len x embedding_dim
        s = self.embedding(s)

        # run the LSTM along the sentences of length seq_len
        # dim: batch_size x seq_len x lstm_hidden_dim
        s, _ = self.lstm(s)

        # make the Variable contiguous in memory (a PyTorch artefact)
        s = s.contiguous()

        # reshape the Variable so that each row contains one token
        # dim: batch_size*seq_len x lstm_hidden_dim
        s = s.view(-1, s.shape[2])

        # apply the fully connected layer and obtain the output (before softmax) for each token
        s = self.fc(s)                   # dim: batch_size*seq_len x num_tags

        # apply log softmax on each token's output (this is recommended over applying softmax
        # since it is numerically more stable)
        return F.log_softmax(s, dim=1)   # dim: batch_size*seq_len x num_tags

In [12]:
def loss_fn(outputs, labels):
    """
    Compute the cross entropy loss given outputs from the model and labels for all tokens. Exclude loss terms
    for PADding tokens.

    Args:
        outputs: (Variable) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (Variable) dimension batch_size x seq_len where each element is either a label in [0, 1, ... num_tag-1],
                or -1 in case it is a PADding token.

    Returns:
        loss: (Variable) cross entropy loss for all tokens in the batch

    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0).float()

    # indexing with negative values is not supported. Since PADded tokens have label -1, we convert them to a positive
    # number. This does not affect training, since we ignore the PADded tokens with the mask.
    labels = labels % outputs.shape[1]

    num_tokens = int(torch.sum(mask))

    # compute cross entropy loss for all tokens (except PADding tokens), by multiplying with mask.
    return -torch.sum(outputs[range(outputs.shape[0]), labels]*mask)/num_tokens

In [13]:
class RunningAverage:
    """A simple class that maintains the running average of a quantity

    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """

    def __init__(self):
        self.steps = 0
        self.total = 0

    def update(self, val):
        self.total += val
        self.steps += 1

    def __call__(self):
        return self.total / float(self.steps)

In [14]:
def train(model, optimizer, loss_fn, data_iterator, metrics, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch, _ = next(data_iterator)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % 10 == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    print("- Train metrics: " + metrics_string)

In [15]:
def evaluate(model, loss_fn, data_iterator, metrics, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch, _ = next(data_iterator)

        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    print(f"summ: {summ}")
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    return metrics_mean

In [16]:
def train_and_evaluate(
        model,
        train_sentences,
        train_labels,
        val_sentences,
        val_labels,
        num_epochs: int,
        batch_size: int,
        optimizer,
        loss_fn,
        metrics
):
    for epoch in range(num_epochs):
        # Run one epoch
        print("Epoch {}/{}".format(epoch + 1, num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (len(train_sentences) + 1) // batch_size
        train_data_iterator = data_iterator(
            train_sentences, train_labels, len(train_sentences), batch_size, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator,
              metrics, num_steps)

        # Evaluate for one epoch on validation set
        num_steps = (len(val_sentences) + 1) // batch_size
        val_data_iterator = data_iterator(
            val_sentences, val_labels, len(val_sentences), batch_size, shuffle=False)
        val_metrics = evaluate(
            model, loss_fn, val_data_iterator, metrics, num_steps)
   

In [17]:
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all tokens. Exclude PADding terms.

    Args:
        outputs: (np.ndarray) dimension batch_size*seq_len x num_tags - log softmax output of the model
        labels: (np.ndarray) dimension batch_size x seq_len where each element is either a label in
                [0, 1, ... num_tag-1], or -1 in case it is a PADding token.

    Returns: (float) accuracy in [0,1]
    """

    # reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.ravel()

    # since PADding tokens have label -1, we can generate a mask to exclude the loss from those terms
    mask = (labels >= 0)

    # np.argmax gives us the class predicted for each token by the model
    outputs = np.argmax(outputs, axis=1)


    # compare outputs with labels and divide by number of tokens (excluding PADding tokens)
    return np.sum(outputs == labels)/float(np.sum(mask))

import numpy as np
from sklearn.metrics import f1_score

def calculate_multiclass_f1_score(outputs, labels):
    
    labels = labels.ravel()
    mask = (labels >= 0)  
    outputs = np.argmax(outputs, axis=1)
    outputs = outputs[mask]
    labels = labels[mask]
    f1= f1_score(labels, outputs, average='micro')
    return f1
def calculate_multiclass_f1_score2(outputs, labels):
    
    labels = labels.ravel()
    mask = (labels >= 0)  
    outputs = np.argmax(outputs, axis=1)
    outputs = outputs[mask]
    labels = labels[mask]
    f1= f1_score(labels, outputs, average='macro')
    return f1


metrics = {
    'f1_score': calculate_multiclass_f1_score,
    'f1 macro': calculate_multiclass_f1_score2,
    'accuracy': accuracy
    # could add more metrics such as accuracy for each token type
}

In [18]:
# manually change vocab size (unique no. of words) and change label size (unique no. of labels) for now
model = Net(embedding_weights, 300, 300, len(tag_map))
optimizer = optim.Adam(model.parameters(), lr=0.001)


if (os.path.isfile("model_weights.pth")):
    model.load_state_dict(torch.load('model_weights.pth'))
else:
    train_and_evaluate(model, train_sentences, train_labels, val_sentences, val_labels, 10, 5, optimizer, loss_fn, metrics)
    torch.save(model.state_dict(), 'model_weights.pth')


Epoch 1/10


100%|██████████| 2808/2808 [00:54<00:00, 51.11it/s, loss=0.178]


- Train metrics: f1_score: 0.946 ; f1 macro: 0.763 ; accuracy: 0.946 ; loss: 0.182
summ: [{'f1_score': 0.975609756097561, 'f1 macro': 0.8615734989648033, 'accuracy': 0.975609756097561, 'loss': 0.08081556856632233}, {'f1_score': 0.9821428571428571, 'f1 macro': 0.9146911952700544, 'accuracy': 0.9821428571428571, 'loss': 0.10781160742044449}, {'f1_score': 0.9857142857142858, 'f1 macro': 0.8476480284011665, 'accuracy': 0.9857142857142858, 'loss': 0.06814499944448471}, {'f1_score': 0.9545454545454546, 'f1 macro': 0.6666666666666666, 'accuracy': 0.9545454545454546, 'loss': 0.2088136225938797}, {'f1_score': 0.941747572815534, 'f1 macro': 0.7669657351576994, 'accuracy': 0.941747572815534, 'loss': 0.152922123670578}, {'f1_score': 0.944954128440367, 'f1 macro': 0.5533692722371968, 'accuracy': 0.944954128440367, 'loss': 0.22380508482456207}, {'f1_score': 0.9411764705882353, 'f1 macro': 0.8492647058823529, 'accuracy': 0.9411764705882353, 'loss': 0.2266296148300171}, {'f1_score': 0.9565217391304348

100%|██████████| 2808/2808 [00:51<00:00, 54.58it/s, loss=0.102]


- Train metrics: f1_score: 0.972 ; f1 macro: 0.863 ; accuracy: 0.972 ; loss: 0.098
summ: [{'f1_score': 0.975609756097561, 'f1 macro': 0.8573099415204679, 'accuracy': 0.975609756097561, 'loss': 0.08510977029800415}, {'f1_score': 0.9880952380952381, 'f1 macro': 0.9670721476510067, 'accuracy': 0.9880952380952381, 'loss': 0.09632628411054611}, {'f1_score': 0.9857142857142858, 'f1 macro': 0.8476480284011665, 'accuracy': 0.9857142857142858, 'loss': 0.07305401563644409}, {'f1_score': 0.9545454545454546, 'f1 macro': 0.8298122065727699, 'accuracy': 0.9545454545454546, 'loss': 0.17262102663516998}, {'f1_score': 0.9611650485436893, 'f1 macro': 0.88265837585034, 'accuracy': 0.9611650485436893, 'loss': 0.11517125368118286}, {'f1_score': 0.944954128440367, 'f1 macro': 0.5533692722371968, 'accuracy': 0.944954128440367, 'loss': 0.25828424096107483}, {'f1_score': 0.9411764705882353, 'f1 macro': 0.8492647058823529, 'accuracy': 0.9411764705882353, 'loss': 0.22564882040023804}, {'f1_score': 0.956521739130

...

- Train metrics: f1_score: 0.993 ; f1 macro: 0.964 ; accuracy: 0.993 ; loss: 0.025
summ: [{'f1_score': 0.959349593495935, 'f1 macro': 0.6851461988304093, 'accuracy': 0.959349593495935, 'loss': 0.144569531083107}, {'f1_score': 0.9821428571428571, 'f1 macro': 0.9200636006424597, 'accuracy': 0.9821428571428571, 'loss': 0.162666916847229}, {'f1_score': 0.9857142857142858, 'f1 macro': 0.8476480284011665, 'accuracy': 0.9857142857142858, 'loss': 0.14031186699867249}, {'f1_score': 0.9545454545454546, 'f1 macro': 0.8298122065727699, 'accuracy': 0.9545454545454546, 'loss': 0.10742898285388947}, {'f1_score': 0.970873786407767, 'f1 macro': 0.9234850108022211, 'accuracy': 0.970873786407767, 'loss': 0.06054769083857536}, {'f1_score': 0.944954128440367, 'f1 macro': 0.6403309209633286, 'accuracy': 0.944954128440367, 'loss': 0.33158648014068604}, {'f1_score': 0.8529411764705882, 'f1 macro': 0.5159313725490196, 'accuracy': 0.8529411764705882, 'loss': 0.45672550797462463}, {'f1_score': 0.9347826086956522

100%|██████████| 2808/2808 [00:55<00:00, 50.65it/s, loss=0.025]


- Train metrics: f1_score: 0.993 ; f1 macro: 0.966 ; accuracy: 0.993 ; loss: 0.022
summ: [{'f1_score': 0.967479674796748, 'f1 macro': 0.7200000000000001, 'accuracy': 0.967479674796748, 'loss': 0.1137431338429451}, {'f1_score': 0.9880952380952381, 'f1 macro': 0.9670721476510067, 'accuracy': 0.9880952380952381, 'loss': 0.1291869431734085}, {'f1_score': 0.9928571428571429, 'f1 macro': 0.9324965132496512, 'accuracy': 0.9928571428571429, 'loss': 0.0972670316696167}, {'f1_score': 0.9545454545454546, 'f1 macro': 0.849551414768806, 'accuracy': 0.9545454545454546, 'loss': 0.15196551382541656}, {'f1_score': 0.9805825242718447, 'f1 macro': 0.9613997113997114, 'accuracy': 0.9805825242718447, 'loss': 0.059913069009780884}, {'f1_score': 0.944954128440367, 'f1 macro': 0.6403309209633286, 'accuracy': 0.944954128440367, 'loss': 0.3813822269439697}, {'f1_score': 0.9117647058823528, 'f1 macro': 0.6468837535014005, 'accuracy': 0.9117647058823529, 'loss': 0.2512919306755066}, {'f1_score': 0.934782608695652

100%|██████████| 2808/2808 [00:57<00:00, 48.49it/s, loss=0.022]


- Train metrics: f1_score: 0.994 ; f1 macro: 0.973 ; accuracy: 0.994 ; loss: 0.020
summ: [{'f1_score': 0.959349593495935, 'f1 macro': 0.6851461988304093, 'accuracy': 0.959349593495935, 'loss': 0.12565183639526367}, {'f1_score': 0.9821428571428571, 'f1 macro': 0.9200636006424597, 'accuracy': 0.9821428571428571, 'loss': 0.13334199786186218}, {'f1_score': 0.9857142857142858, 'f1 macro': 0.8476480284011665, 'accuracy': 0.9857142857142858, 'loss': 0.13992397487163544}, {'f1_score': 0.9545454545454546, 'f1 macro': 0.849551414768806, 'accuracy': 0.9545454545454546, 'loss': 0.09906823933124542}, {'f1_score': 0.9902912621359223, 'f1 macro': 0.9753042233357194, 'accuracy': 0.9902912621359223, 'loss': 0.047607745975255966}, {'f1_score': 0.944954128440367, 'f1 macro': 0.6403309209633286, 'accuracy': 0.944954128440367, 'loss': 0.38082778453826904}, {'f1_score': 0.8823529411764706, 'f1 macro': 0.5754551820728291, 'accuracy': 0.8823529411764706, 'loss': 0.43476995825767517}, {'f1_score': 0.9130434782

In [19]:
print(len(test_sentences))
test_data_iterator = data_iterator(test_sentences, test_labels, len(test_sentences), len(test_sentences), shuffle=True)
test_batch, labels_batch, test_sentences = next(test_data_iterator)



3453


In [30]:
print(f"type test: {type(test_batch)}")
print(f"len test: {len(test_batch)}")
print(f"type label: {type(labels_batch)}")
print(f"len label: {len(labels_batch)}")

type test: <class 'torch.Tensor'>
len test: 3453
type label: <class 'torch.Tensor'>
len label: 3453


In [28]:
model_output = model(test_batch)

In [29]:
len(model_output)

428172

In [33]:
model_output_numpy = model_output.detach().numpy()
labels_batch_numpy = labels_batch.detach().numpy()

In [40]:
f1_score_micro = calculate_multiclass_f1_score(model_output_numpy, labels_batch_numpy)
f1_score_macro = calculate_multiclass_f1_score2(model_output_numpy, labels_batch_numpy)

In [43]:
inv_vocab = {v: k for k, v in vocab.items()}
inv_tag_map = {v: k for k, v in tag_map.items()}

In [72]:
def id_to_words(sentence):
    new_sentence = [inv_vocab[i] for i in sentence]
    return new_sentence

def id_to_labels(labels):
    new_sentence = [inv_tag_map[i] for i in labels]
    return new_sentence


In [53]:
predicted_labels = np.argmax(model_output.detach().numpy(), axis=1)
print(f"sentences_w_words \n {len(test_sentences)}")
print(f"predicted_labels \n {len(predicted_labels)}")

sentences_w_words 
 3453
predicted_labels 
 428172


In [87]:
sample_output = model(test_batch[10].unsqueeze(0))
id_to_words(test_sentences[10])


['Yevgeny',
 'Kafelnikov',
 'UNK',
 'Russia',
 'UNK',
 'beat',
 'Jim',
 'Courier',
 'UNK',
 'U.S.',
 'UNK',
 'UNK',
 'UNK',
 'UNK']

In [88]:
sample_mask = (labels_batch[10] >= 0)
sample_label_predict = np.argmax(sample_output.detach().numpy(), axis=1)[sample_mask]
sample_label_true = labels_batch[10][sample_mask]

In [89]:
print(f"sample_label_predict: {id_to_labels(sample_label_predict)}")
print(f"sample_label_true: {id_to_labels(sample_label_true.numpy())}")

sample_label_predict: ['I-PER', 'I-PER', 'O', 'I-LOC', 'O', 'O', 'I-PER', 'I-PER', 'O', 'I-LOC', 'O', 'O', 'O', 'O']
sample_label_true: ['I-PER', 'I-PER', 'O', 'I-LOC', 'O', 'O', 'I-PER', 'I-PER', 'O', 'I-LOC', 'O', 'O', 'O', 'O']
